In [1]:
import os
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '0'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
from huggingface_hub import login, whoami

login()

In [2]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

43

In [3]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-09 15:01:20 __init__.py:190] Automatically detected platform cuda.


In [ ]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    dtype = None,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.5: Fast Gemma patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: vLLM loading unsloth/gemma-2b-it-bnb-4bit with actual GPU utilization = 79.11%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.68 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 16.57 GB. Also swap space = 6 GB.
INFO 02-09 15:01:30 config.py:542] This model supports multiple tasks: {'embed', 'classify', 'generate', 'score', 'reward'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_qu

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-09 15:08:57 model_runner.py:1115] Loading model weights took 1.9409 GB
INFO 02-09 15:08:57 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-09 15:08:58 worker.py:267] Memory profiling takes 1.01 seconds
INFO 02-09 15:08:58 worker.py:267] the current vLLM instance can use total_gpu_memory (23.68GiB) x gpu_memory_utilization (0.79) = 18.74GiB
INFO 02-09 15:08:58 worker.py:267] model weights take 1.94GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 2.66GiB; the rest of the memory reserved for KV Cache is 14.08GiB.
INFO 02-09 15:08:59 executor_base.py:110] # CUDA blocks: 51249, # CPU blocks: 21845
INFO 02-09 15:08:59 executor_base.py:115] Maximum concurrency for 512 tokens per request: 1601.53x
INFO 02-09 15:09:02 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory erro

Capturing CUDA graph shapes: 100%|██████████| 39/39 [00:19<00:00,  1.99it/s]

INFO 02-09 15:09:22 model_runner.py:1562] Graph capturing finished in 20 secs, took 0.58 GiB
INFO 02-09 15:09:22 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 24.62 seconds



Unsloth 2025.2.5 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [12]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}<end_of_turn>\n"
BOS_TOKEN = "<bos>"  # Assuming a beginning of sequence token is needed

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'user', 'content': SYSTEM_PROMPT},
            {'role': 'assistant', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [13]:
dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72',
 'prompt': [{'content': '\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
   'role': 'user'},
  {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
   'role': 'assistant'}]}

In [17]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


In [18]:
CHAT_TEMPLATE = """
{%- for message in messages %}
    {{- '<|' + message['role'] + '|>\n' }}
    {{- message['content'] + eos_token }}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|assistant|>\n' }}
{%- endif %}
"""

if tokenizer.chat_template is None:
    tokenizer.chat_template = CHAT_TEMPLATE # set the attribute

In [19]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 19,611,648


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
... Tina had 24 brownies.

... She had one with lunch and dinner every day for 5 days, which is 20 servings.

... She had 5 pieces left for dinner on 5 different days. 
Extracted:
... Tina had 24 brownies.

... She had one with lunch and dinner every day for 5 days, which is 20 servings.

... She had 5 pieces left for dinner on 5 different days.


Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.000000,0.000000,26.166668,0.000000
2,0.000000,0.520833,0.320319,45.666668,0.000000
3,0.000000,0.270833,0.255155,40.000000,0.000465
4,0.000000,0.000000,0.000000,81.000000,0.000177
5,0.000000,0.000000,0.000000,36.166668,0.000506
6,0.000000,0.000000,0.000000,70.333336,0.000519
7,0.000000,0.000000,0.000000,48.833336,0.000211
8,0.000000,0.250000,0.387298,74.833336,0.000354
9,0.000000,0.000000,0.000000,39.000000,0.000334
10,0.000000,0.000000,0.000000,30.833334,0.000463


-------------------- Question:
The gauge on a water tank shows that the tank is 1/3 full of water. To fill the tank, 16 gallons of water are added. How many gallons of water does the tank hold when full? 
Answer:
24 
Response:
<reasoning>
The tank holds 1 + 16 = 17 gallons of water when full.
</reasoning>
<answer>17 gallons</answer> 
Extracted:
17 gallons
-------------------- Question:
There are 15 tables in the school's cafeteria. Each table can seat 10 people. Usually, only 1/10 of the seats are left unseated. How many seats are usually taken? 
Answer:
135 
Response:
Reasoning: 15 tables * 10 seats/table = 150 seats.
Answer: 150 seats are usually taken in the school's cafeteria. 
Extracted:
Reasoning: 15 tables * 10 seats/table = 150 seats.
Answer: 150 seats are usually taken in the school's cafeteria.
-------------------- Question:
Fiona completed 36 math questions in an hour. Shirley was able to complete twice as many math questions within that same time, and Kiana completed half o

TrainOutput(global_step=250, training_loss=0.0025955065527729746, metrics={'train_runtime': 553.8949, 'train_samples_per_second': 0.451, 'train_steps_per_second': 0.451, 'total_flos': 0.0, 'train_loss': 0.0025955065527729746})

In [20]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

print(output)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it, est. speed input: 9.11 toks/s, output: 165.32 toks/s]

Pi can be calculated using various methods, but the most common and widely accepted one is through a geometric series:

**Pi = 3 + 1/3 + 1/9 + 1/27 + ...**

This series converges to pi. However, it doesn't converge to pi exactly, but to the next digit.

Here are some other methods for calculating pi:

* **Using the circumference of a circle:** The circumference of a circle is 2πr, where r is the radius. We can then find pi by dividing the circumference by the diameter.
* **Using the area of a circle:** The area of a circle is πr², where r is the radius.
* **Using a calculator or computer:** There are many online and mobile calculators that can provide pi values.
* **Using the infinite series:** Pi can also be expressed as an infinite series, but it's too cumbersome to use for most practical purposes.

The most accurate way to calculate pi is through computation. However, you can use the geometric series method to get an approximate value and then refine it using other methods as needed

In [21]:
model.save_lora("grpo_gemma_saved_lora")

In [44]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : SYSTEM_PROMPT},
    {"role" : "assistant", "content" : "which is greater: 1.9 or 1.11"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_gemma_saved_lora"),
)[0].outputs[0].text

print(output)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.45it/s, est. speed input: 139.79 toks/s, output: 139.79 toks/s]

<reasoning>
1.9 is greater than 1.11. 1.9 is closer to 2 than 1.11, which is closer to 1. 
</reasoning>
<answer>1.9</answer>


In [ ]:
model.save_pretrained_merged("unsloth_gemma-2-2b-r1-reasoning", tokenizer, save_method = "merged_16bit",)
# model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 75.39 out of 124.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 18/18 [00:02<00:00,  6.62it/s]


Unsloth: Saving tokenizer... Done.
Done.
